In [1]:
from torchvision import models,transforms,datasets
import torch
from PIL import Image
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torch
import time
import copy
from torch.optim import lr_scheduler
from torchsummary import summary
%matplotlib inline

In [2]:
use_gpu = torch.cuda.is_available()
if (use_gpu): print ('GPU: True')
else : print('GPU: False')
print ('Torch version : ', torch.__version__)

use_gpu = False

GPU: True
Torch version :  1.0.0


### Data Loaders

In [3]:
'''
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], 
                             std= [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])
    ]),
}
'''
####### this is an example ############

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(256),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(degrees=90),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)]),
        }

In [4]:
data_dir = 'data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=1,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

print ('total classes:', len(class_names))
print (class_names)

total classes: 7
['go', 'goForward', 'goLeft', 'stop', 'stopLeft', 'warning', 'warningLeft']


### Model Construction

In [5]:
cgf = [64, 64, 'M', 
       128, 128, 'M', 
       256, 256, 256, 'M', 
       512, 512, 512, 'M',
       512, 512, 512, 'M']

In [6]:
class BullyNet(nn.Module):
 
    def __init__(self, cfg, inchannels, num_classes = 10, init_weights= True, batch_norm= True):
        super(BullyNet, self).__init__()
        layers = []
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            elif v == 'C':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
            else:
                conv2d = nn.Conv2d(inchannels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                inchannels = v
        
        pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
        conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
        
        layers += [conv6, nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
        
        self.features = nn.Sequential(*layers)
        self.classifier = nn.Sequential(
            nn.Linear(1024*7*7, 4096),
            nn.ReLU(True), # inplace=True
            nn.Dropout(),
            nn.Linear(4096,256),
            nn.ReLU(True),
            nn.Dropout(), # p=0.5,inplace=False
            nn.Linear(256,128),
            nn.ReLU(True),
            nn.Dropout(), # p=0.5,inplace=False
            nn.Linear(128,num_classes),
        )
        
    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x

In [7]:
vgg16 = BullyNet(cgf, inchannels=3, num_classes=len(class_names))

In [8]:
print (vgg16)

BullyNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(

In [9]:
#model_vgg = vgg16.cuda()
model_vgg = vgg16

### Training

In [10]:
loss_func = nn.CrossEntropyLoss()
optimizer= optim.SGD(model_vgg.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [11]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{} : '.format(epoch, num_epochs - 1), end = '\t\t')
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if(phase == 'train'):
                print('{} Loss: {:.4f} \tAcc: {:.4f}'.format(phase, epoch_loss, epoch_acc), end = '\t\t')
            else : 
                print('{} Loss: {:.4f} \tAcc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                
            
            if(phase=='train'):
                train_acc.append(epoch_acc)
                train_loss.append(epoch_loss)
            else:
                val_acc.append(epoch_acc)
                val_loss.append(epoch_loss)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
vgg16 = train_model(model_vgg, 
                    loss_func, 
                    optimizer, 
                    exp_lr_scheduler, 
                    num_epochs=1000)

Epoch 0/999 : 		

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.show()

plt.plot(train_acc)
plt.plot(val_acc)
plt.show()

In [ ]:
torch.save(vgg16.state_dict(), 'models/VGG16_without_init_weights_3_7.pt')

In [ ]:
print ('Done!')